<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-15 11:45:05
-------------------
qualified stocks: 95
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  2.21 L
Current:  1.26 C
-------------------
Today PnL: 38.62 K (0.31%)
Current PnL: -17.69 L (-12.55%)
CY Booked + Current PnL: -14.83 L (-10.51%)
-------------------
Total profit:  3.80 L
Total loss:  -21.50 L
-------------------
Total Booked + Current PnL: 12.15 L (10.66%)
Total Booked PnL: 29.85 L (26.17%)
Curr Year Booked PnL: 2.87 L (2.28%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.06 C
Est FTT PnL: 80.16 L (63.65%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.42 C (112.53%)
Deployed:  1.14 C
Current:  1.26 C
CAGR/XIRR %: 7.15%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.12,15.30,X-MC,90.53,141275.0,26497.0,9225.0,0.15,23.09,6.53,31.13,220.0,2.87,1.14,37.62,AR,ATH,TYRES
56,PIDILITIND,2504.06,3576.00,-15.46,X-LC,44.21,93669.0,18547.0,13610.0,0.44,24.69,14.53,42.81,120.0,1.36,0.76,17.47,X40,BTT,CHEMICALS
69,SBILIFE,1496.49,1924.99,-0.75,H-LC,82.11,147428.0,21723.0,14271.0,-0.01,17.28,9.68,28.63,147.0,1.52,1.19,31.34,AR,ATH,INSURANCE
62,RELIANCE,1288.53,1526.00,4.61,X-LC,88.42,224866.0,21278.0,16235.0,-0.08,10.45,7.22,18.43,157.0,1.31,1.82,22.49,XY25,NTT,REFINERIES
70,SBIN,760.30,863.00,-13.45,M-LC,43.16,210940.0,9460.0,17761.0,-0.59,4.70,8.42,13.51,60.0,0.53,1.70,15.56,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VALIANTORG,512.64,838.00,-308.98,H-SC,10.53,88790.0,-44496.0,129092.0,4.48,-33.38,145.39,63.47,161.0,-0.34,0.72,48.83,XR,NTT,CHEMICALS
91,VIPIND,488.80,718.00,-818.46,H-SC,100.00,68474.0,-26842.0,71535.0,3.95,-28.16,104.47,46.89,82.0,-0.38,0.55,35.06,X40N,NTT,MISC
5,ALKYLAMINE,2347.98,4567.52,-18.14,X-SC,21.05,76132.0,-24831.0,120273.0,2.59,-24.59,157.98,94.53,70.0,-0.21,0.62,12.87,SR,ATH,CHEMICALS
59,RAJESHEXPO,517.65,583.00,1833.08,M-SC,40.00,52730.0,-84447.0,101764.0,2.53,-61.56,192.99,12.62,195.0,-0.83,0.43,31.14,X40N,NTT,JEWELLERY
81,TATAMOTORS,776.02,1065.00,-52.64,M-LC,93.68,189846.0,-15799.0,92379.0,2.50,-7.68,48.66,37.24,1.0,-0.17,1.53,23.57,XY24,NTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UNIONBANK,123.87,163.00,-14.11,M-LC,62.11,150255.0,9415.0,35070.0,-2.06,6.68,23.34,31.59,48.0,0.27,1.21,30.52,XY24,NTT,BANKS
58,QUESS,711.66,986.00,NaN,NaN,97.89,79081.0,-76773.0,136850.0,-1.94,-49.26,173.05,38.55,NaN,-0.56,0.64,21.09,XY24,NTT,MISC
79,TANLA,917.30,1963.11,-40.61,H-SC,95.79,145651.0,-93764.0,366720.0,-1.71,-39.16,251.78,114.01,20.0,-0.26,1.18,33.58,AR,ATH,IT
71,SFL,1016.35,1287.00,274.08,H-SC,25.26,168822.0,-93396.0,163217.0,-1.39,-35.62,96.68,26.63,58.0,-0.57,1.36,1.59,XY24,NTT,MISC
39,INDUSINDBK,1354.50,1800.00,-40.59,H-MC,47.37,43930.0,-33276.0,58669.0,-1.36,-43.10,133.55,32.89,24.0,-0.57,0.36,20.98,XR,NTT,BANKS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TATAMOTORS,776.02,1065.00,-52.64,M-LC,93.68,189846.0,-15799.0,92379.0,2.50,-7.68,48.66,37.24,1.0,-0.17,1.53,23.57,XY24,NTT,AUTO
37,INDIAMART,2311.97,4911.36,-52.59,H-SC,69.47,102608.0,881.0,113495.0,2.11,0.87,110.61,112.43,2.0,0.01,0.83,21.10,AR,ATH,MISC
2,ABB,5319.37,8953.12,-37.83,H-LC,68.42,142238.0,9254.0,81588.0,0.97,6.96,57.36,68.31,4.0,0.11,1.15,15.98,AR,ATH,ELECTRICAL
82,TCS,3928.29,4998.00,-15.52,X-LC,66.32,190706.0,-21422.0,79181.0,-0.48,-10.10,41.52,27.23,5.0,-0.27,1.54,9.29,X40,BTT,IT
48,LICI,777.74,983.00,-16.04,H-LC,52.63,186346.0,12132.0,33840.0,1.18,6.96,18.16,26.39,6.0,0.36,1.51,14.08,XY25,NTT,INSURANCE


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.12,15.30,X-MC,90.53,141275.0,26497.0,9225.0,0.15,23.09,6.53,31.13,220.0,2.87,1.14,37.62,AR,ATH,TYRES
52,MEDANTA,1022.60,1486.00,7.25,X-MC,13.68,189134.0,33699.0,36730.0,0.11,21.68,19.42,45.32,201.0,0.92,1.53,28.69,XY24,NTT,HEALTHCARE
56,PIDILITIND,2504.06,3576.00,-15.46,X-LC,44.21,93669.0,18547.0,13610.0,0.44,24.69,14.53,42.81,120.0,1.36,0.76,17.47,X40,BTT,CHEMICALS
57,POLYCAB,5005.92,7541.35,8.17,H-MC,92.11,169120.0,28954.0,42043.0,0.99,20.66,24.86,50.65,55.0,0.69,1.37,29.07,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-1.85,M-LC,87.37,190019.0,38813.0,26850.0,1.11,25.67,14.13,43.43,37.0,1.45,1.54,31.71,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LUXIND,1448.19,2442.00,33.16,M-SC,26.32,137845.0,267.0,94148.0,-0.34,0.19,68.30,68.62,79.0,0.00,1.11,19.50,X40N,NTT,TEXTILES
57,POLYCAB,5005.92,7541.35,8.17,H-MC,92.11,169120.0,28954.0,42043.0,0.99,20.66,24.86,50.65,55.0,0.69,1.37,29.07,X40N,ATH,CABLES
6,ANGELONE,2328.67,3033.00,2.48,H-SC,77.89,167115.0,15751.0,30031.0,0.87,10.41,17.97,30.25,111.0,0.52,1.35,32.27,X40N,NTT,FINANCE
4,ADANIPORTS,1103.69,1583.00,-1.85,M-LC,87.37,190019.0,38813.0,26850.0,1.11,25.67,14.13,43.43,37.0,1.45,1.54,31.71,XY24,NTT,MISC
28,GILLETTE,8109.44,10330.69,-4.26,H-SC,41.58,141202.0,3342.0,34425.0,1.13,2.42,24.38,27.39,68.0,0.10,1.14,25.27,X40,ATH,FMCG


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
94,WIPRO,243.46,420.00,-8.80,M-LC,36.84,156383.0,5438.0,104010.0,-0.28,3.60,66.51,72.51,52.0,0.05,1.26,17.74,XR,NTT,IT
18,CAMS,3643.00,5250.99,-8.22,H-SC,7.37,103152.0,1148.0,43881.0,0.96,1.13,42.54,44.14,22.0,0.03,0.83,17.55,XR,ATH,MISC
37,INDIAMART,2311.97,4911.36,-52.59,H-SC,69.47,102608.0,881.0,113495.0,2.11,0.87,110.61,112.43,2.0,0.01,0.83,21.10,AR,ATH,MISC
12,BANKINDIA,113.49,190.00,-34.31,M-MC,31.58,129061.0,-91.0,87155.0,-0.39,-0.07,67.53,67.42,41.0,-0.00,1.04,25.52,XR,NTT,BANKS
92,VOLTAS,1272.00,1929.20,-31.02,H-MC,12.63,137775.0,-2145.0,74440.0,0.32,-1.53,54.03,51.67,32.0,-0.03,1.11,4.06,AR,ATH,AC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,UJJIVANSFB,52.77,60.00,38.86,M-SC,78.95,120042.0,-22437.0,41955.0,-0.98,-15.75,34.95,13.70,214.0,-0.53,0.97,41.68,X40N,NTT,BANKS
53,MRF,114777.60,150504.12,15.30,X-MC,90.53,141275.0,26497.0,9225.0,0.15,23.09,6.53,31.13,220.0,2.87,1.14,37.62,AR,ATH,TYRES
69,SBILIFE,1496.49,1924.99,-0.75,H-LC,82.11,147428.0,21723.0,14271.0,-0.01,17.28,9.68,28.63,147.0,1.52,1.19,31.34,AR,ATH,INSURANCE
87,UNIONBANK,123.87,163.00,-14.11,M-LC,62.11,150255.0,9415.0,35070.0,-2.06,6.68,23.34,31.59,48.0,0.27,1.21,30.52,XY24,NTT,BANKS
14,BERGEPAINT,558.54,680.00,-16.80,H-MC,60.00,178693.0,1636.0,36864.0,1.94,0.92,20.63,21.75,154.0,0.04,1.44,27.82,XY24,NTT,PAINTS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,VIPIND,488.80,718.00,-818.46,H-SC,100.00,68474.0,-26842.0,71535.0,3.95,-28.16,104.47,46.89,82.0,-0.38,0.55,35.06,X40N,NTT,MISC
80,TATAELXSI,7332.28,9000.00,-18.33,X-MC,98.95,85792.0,-16860.0,40211.0,0.45,-16.42,46.87,22.74,47.0,-0.42,0.69,28.95,X40N,NTT,IT
58,QUESS,711.66,986.00,NaN,NaN,97.89,79081.0,-76773.0,136850.0,-1.94,-49.26,173.05,38.55,NaN,-0.56,0.64,21.09,XY24,NTT,MISC
1,AARTIIND,487.04,919.00,35.69,M-SC,96.84,77889.0,-3447.0,75583.0,0.52,-4.24,97.04,88.69,159.0,-0.05,0.63,31.34,XR,NTT,CHEMICALS
79,TANLA,917.30,1963.11,-40.61,H-SC,95.79,145651.0,-93764.0,366720.0,-1.71,-39.16,251.78,114.01,20.0,-0.26,1.18,33.58,AR,ATH,IT


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.41
1,25,39.20
2,50,68.22


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    35.03
X40N    15.34
X40     14.47
AR      14.07
XR      10.38
XY25     8.35
X200     1.21
SR       1.13
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.56
M-SC    17.87
X-LC    16.34
M-LC     8.39
H-MC     7.90
X-MC     6.78
H-LC     5.93
M-MC     5.57
X-SC     2.84
L-SC     2.29
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.76,-1.85,34.25
BANKS,11.08,-9.78,63.63
IT,9.27,-20.54,87.79
FINANCE,7.49,-19.43,73.32
MISC,7.21,-26.90,81.50
INSURANCE,6.12,-2.14,32.93
PAINTS,5.62,-18.20,53.18
ELECTRICAL,4.14,-8.61,63.95
HEALTHCARE,3.66,-3.00,36.25


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3015555.0,27
X40N,1265537.0,18
XR,1260629.0,15
AR,1260542.0,15
X40,804065.0,12
SR,206044.0,2
XY25,135600.0,5
X200,67831.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2398226.0,22
M-SC,2027110.0,21
X-LC,718579.0,12
M-MC,577445.0,5
H-MC,420204.0,7
X-SC,358143.0,4
M-LC,339879.0,6
L-SC,315168.0,4
X-MC,314950.0,6


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      917735.0      6
M-SC       XY24      765685.0      6
H-SC       X40N      547409.0      7
           AR        541070.0      3
M-SC       X40N      452495.0      6
           XR        429533.0      5
M-MC       XY24      405725.0      3
X-LC       X40       378531.0      5
H-SC       XR        298364.0      4
M-LC       XY24      218108.0      4
H-LC       AR        204663.0      4
M-SC       X40       199938.0      2
X-LC       AR        188218.0      3
H-MC       XY24      173416.0      2
L-SC       XR        162953.0      2
X-MC       XY24      139748.0      2
X-SC       SR        120273.0      1
           XR        119945.0      1
           X40N      117925.0      2
X-MC       X40N      105665.0      2
M-LC       XR        104010.0      1
M-SC       AR         93688.0      1
H-SC       X40        93648.0      2
L-SC       XY24       87542.0      1
M-MC       XR         87155.0      1
M-SC       SR         85771.0      1
M-MC       AR         84565.0      1
X-LC       XY25       83999.0      3
H-MC       AR         74440.0      1
           X40        71636.0      2
X-LC       X200       67831.0      1
L-SC       AR         64673.0      1
X-MC       X40        60312.0      1
H-MC       XR         58669.0      1
           X40N       42043.0      1
H-LC       XY25       33840.0      1
M-LC       XY25       17761.0      1
X-MC       AR          9225.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
